#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [3]:
import graphlab

A newer version of GraphLab Create (v1.7.1) is available! Your current version is v1.6.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


#Read some product review data

Loading reviews for a set of baby products. 

In [4]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to umenon@acm.org and will expire on November 09, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-3836 - Server binary: C:\Users\Room2\Anaconda2\envs\dato-env\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Room2\AppData\Local\Temp\graphlab_server_1448132320.log.0
[INFO] GraphLab Server Version: 1.6.1


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [5]:
products.head()

Columns:
	name	str
	review	str
	rating	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
[10 rows x 3 columns]

#Build the word count vector for each review

In [6]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [5]:
products.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+
|           word_count          |
+-------------------------------+
| {'and': 5, 'stink': 1, 'be... |
| {'and': 3, 'love': 1, 'it'... |
| {'and': 2, 'quilt': 1, 'it... |
| {'ingenious': 1, 'and': 3,... |
| {'and': 2, 'parents!!': 1,... |
| {'and': 2, 'cute': 1, 'hel... |
| {'shop': 1, 'be': 1, 'is':... |
| {'feeding,': 1, 'and': 2, ... |
| {'and': 1, 'help': 1, 'giv... |
| {'journal.': 1, 'all': 1, ... |
+-------------------------------+
[10 rows x 4 columns]

In [8]:
graphlab.canvas.set_target('ipynb')

In [9]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [10]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [11]:
len(giraffe_reviews)

785

In [12]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [13]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [14]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [15]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [15]:
products.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|           word_count          | sentiment |
+-------------------------------+-----------+
| {'and': 3L, 'love': 1L, 'i... |     1     |
| {'and': 2L, 'quilt': 1L, '... |     1     |
| {'ingenious': 1L, 'and': 3... |     1     |
| {'and': 2L, 'parents!!': 1... |     1     |
| {'and': 2L, 'cute': 1L, 'h... |     1     |
| {'shop': 1L, 'be': 1L, 'is... |     1     |
| {'feeding,': 1L, 'and': 2L... |     1     |
| {'and': 1L, 'help': 1L, 'g... |     1     |
| {'journal.': 1L, 'all': 1L... |     1     |
| {'all': 1L, 'forget': 1L, ... |     1     |
+-------------------------------+-----------+
[10 rows x 5 columns]

##Let's train the sentiment classifier

In [16]:
train_data,test_data = products.random_split(.8, seed=0)

In [18]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.192036     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 3.330069     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 3.778076     | 0.92

#Evaluate the sentiment model

In [19]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.224079639369 | 0.00375053579083 | 27996 | 5324 |
 | 0.0010000000475  | 0.775920360631 |  0.996249464209  | 27996 | 5324 |
 | 0.00200000009499 | 0.735912847483 |  0.995213601943  | 27996 | 5324 |
 | 0.00300000002608 | 0.713185574756 |  0.994606372339  | 27996 | 5324 |
 | 0.00400000018999 | 0.697595792637 |  0.994142020289  | 27996 | 5324 |
 | 0.00499999988824 | 0.686326070624 |  0.993749107015  | 27996 | 5324 |
 | 0.00600000005215 | 0.676558978212 |  0.993249035577  | 27996 | 5324 |
 | 0.00700000021607 | 0.666040570999 |  0.992891841692  | 27996 | 5324 |
 | 0.00800000037998 | 0.655709992487 |  0.992641805972  | 27996 | 5324 |
 | 0.00899999961257 

In [96]:
sentiment_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [20]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [21]:
giraffe_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | He likes chewing on all th... |  5.0   |
| Vulli Sophie the Giraffe T... | My son loves this toy and ... |  5.0   |
| Vulli Sophie the Giraffe T... | There really should be a l... |  1.0   |
| Vulli Sophie the Giraffe T... | All the moms in my moms' g... |  5.0   |
| Vulli Sophie the Giraffe T... | I was a little skeptical o... |  5.0   |
| Vulli Sophie the Giraffe T... | I have been reading about ... |  5.0   |
| Vulli Sophie the Giraffe T... | My neice loves her sophie ... |  5.0   |
| Vulli Sophie the Giraffe T... | What a friendly face!  And... |  5.0   |
| Vulli Sophie the Giraffe T... | We got this just for my so... |  5.0   |
| Vulli Sophie the Giraffe T... | My baby seems to like this... |  3.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------------------+
|           word_count          | predicted_sentiment |
+-------------------------------+---------------------+
| {'and': 1, 'all': 1, 'beca... |    0.999513023521   |
| {'and': 1, 'right': 1, 'he... |    0.999320678306   |
| {'and': 2, 'all': 1, 'late... |    0.013558811687   |
| {'and': 2, 'one!': 1, 'all... |    0.995769474148   |
| {'and': 3, 'all': 1, 'old'... |    0.662374415673   |
| {'and': 6, 'seven': 1, 'al... |    0.999997148186   |
| {'and': 4, 'drooling,': 1,... |    0.989190989536   |
| {'and': 3, 'chew': 1, "don... |    0.999563518413   |
| {'chew': 2, 'because': 1, ... |    0.970160542725   |
| {'and': 2, 'already': 1, '... |    0.195367644588   |
+-------------------------------+---------------------+
[10 rows x 5 columns]

##Sort the reviews based on the predicted sentiment and explore

In [21]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [24]:
giraffe_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | Sophie, oh Sophie, your ti... |  5.0   |
| Vulli Sophie the Giraffe T... | I'm not sure why Sophie is... |  4.0   |
| Vulli Sophie the Giraffe T... | I'll be honest...I bought ... |  4.0   |
| Vulli Sophie the Giraffe T... | We got this little giraffe... |  5.0   |
| Vulli Sophie the Giraffe T... | As a mother of 16month old... |  5.0   |
| Vulli Sophie the Giraffe T... | Sophie the Giraffe is the ... |  5.0   |
| Vulli Sophie the Giraffe T... | Sophie la giraffe is absol... |  5.0   |
| Vulli Sophie the Giraffe T... | My 5-mos old son took to t... |  5.0   |
| Vulli Sophie the Giraffe T... | My nephews and my four kid... |  5.0   |
| Vulli Sophie the Giraffe T... | Never thought I'd see my s... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------------------+
|           word_count          | predicted_sentiment |
+-------------------------------+---------------------+
| {'giggles': 1, 'all': 1, "... |         1.0         |
| {'peace': 1, 'month': 1, '... |    0.999999999703   |
| {'all': 2, 'pops': 1, 'exi... |    0.999999999392   |
| {'all': 2, "don't": 1, '(l... |    0.99999999919    |
| {'cute': 1, 'all': 1, 'rev... |    0.999999998657   |
| {'just': 2, 'both': 1, 'mo... |    0.999999997108   |
| {'and': 5, 'the': 1, 'all'... |    0.999999995589   |
| {'just': 1, 'shape': 2, 'm... |    0.999999995573   |
| {'and': 4, 'chew': 1, 'all... |    0.999999989527   |
| {'giggles': 1, 'all': 1, '... |    0.999999985069   |
+-------------------------------+---------------------+
[10 rows x 5 columns]

##Most positive reviews for the giraffe

In [22]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [23]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [37]:
products[2]['word_count']

{'a': 2L,
 'about': 1L,
 'and': 3L,
 'anything': 1L,
 'approach': 2L,
 'artwork,': 1L,
 'back,': 1L,
 'binky.': 2L,
 'chart': 1L,
 'clever': 1L,
 'daughter': 1L,
 'else': 1L,
 'fairy.': 1L,
 'found': 1L,
 'getting': 1L,
 'has': 1L,
 'have': 1L,
 'her': 1L,
 'herself,': 1L,
 'how': 1L,
 'i': 3L,
 'in': 2L,
 'ingenious': 1L,
 'is': 4L,
 'it': 1L,
 'like': 1L,
 'little': 1L,
 'losing': 1L,
 'love': 2L,
 'loves': 1L,
 'most': 1L,
 'much': 1L,
 'my': 1L,
 'not': 1L,
 'of': 3L,
 'ownership': 1L,
 'positive,': 1L,
 'product': 2L,
 'proud': 1L,
 'purchase.': 1L,
 'rid': 1L,
 'she': 1L,
 'so': 1L,
 'the': 7L,
 'this': 3L,
 'this,': 1L,
 'to': 1L,
 'tool.': 1L,
 'well': 1L,
 'what': 1L,
 'worth': 1L}

In [49]:
def awesome_count (aDict):
    if ('awesome' in aDict):
        return aDict['awesome']
    else:
        return 0


In [55]:
test = {'awesome':3,'foo':2}
awesome_count(test)

3

In [57]:
products['awesome'] = products['word_count'].apply(awesome_count)

In [58]:
products['awesome'].head()

dtype: int
Rows: 10
[0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]

In [61]:
def sel_word_count (aDict):
    retDict = {}
    for selword in selected_words:
        if (selword in aDict):
            retDict[selword] = aDict[selword]
    return retDict


In [83]:
products['sel_word_count'] = products['word_count'].apply(sel_word_count)

In [66]:
products[30]['sel_word_count']

{'great': 1L}

### Using the .sum() method on each of the new columns you created, answer the following questions: Out of the selected_words, which one is most used in the dataset? Which one is least used? Save these results to answer the quiz at the end.

In [69]:
def cum_sel_word_count():
    retDict = {}
    for dict in products['sel_word_count']:
        for k, v in dict.iteritems():
            if k in retDict:
               retDict[k] =  retDict[k] + v
            else:
               retDict[k] = v
    return retDict

In [70]:
cum_sel_word_count()

{'amazing': 1305L,
 'awesome': 2002L,
 'awful': 345L,
 'bad': 3197L,
 'fantastic': 873L,
 'great': 42420L,
 'hate': 1057L,
 'horrible': 659L,
 'love': 40277L,
 'terrible': 673L,
 'wow': 131L}

### Create a new sentiment analysis model using only the selected_words as features

In [71]:
train_data,test_data = products.random_split(.8, seed=0)

In [72]:
features=selected_words

In [73]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['sel_word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.421024     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.631036     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.739043     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        | 

### Using this approach, sort the learned coefficients according to the ‘value’ column using .sort(). Out of the 11 words in selected_words, which one got the most positive weight? Which one got the most negative weight? Do these values make sense for you? Save these results to answer the quiz at the end

In [75]:
selected_words_model['coefficients']

Columns:
	name	str
	index	str
	class	int
	value	float

Rows: 12

Data:
+----------------+-----------+-------+------------------+
|      name      |   index   | class |      value       |
+----------------+-----------+-------+------------------+
|  (intercept)   |    None   |   1   |  1.36728315229   |
| sel_word_count |    love   |   1   |  1.39989834302   |
| sel_word_count |   great   |   1   |  0.883937894898  |
| sel_word_count |  amazing  |   1   |  0.892802422508  |
| sel_word_count | fantastic |   1   |  0.891303090304  |
| sel_word_count |  terrible |   1   |  -2.09049998487  |
| sel_word_count |    bad    |   1   | -0.985827369929  |
| sel_word_count |  awesome  |   1   |  1.05800888878   |
| sel_word_count |    wow    |   1   | -0.0541450123333 |
| sel_word_count |    hate   |   1   |  -1.40916406276  |
+----------------+-----------+-------+------------------+
[12 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

### What is the accuracy of the selected_words_model on the test_data? What was the accuracy of the sentiment_model that we learned using all the word counts in the IPython Notebook above from the lectures? What is the accuracy majority class classifier on this task? How do you compare the different learned models with the baseline approach where we are just predicting the majority class? Save these results to answer the quiz at the end.

In [76]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+-----+-------+------+
 |    threshold     |        fpr        | tpr |   p   |  n   |
 +------------------+-------------------+-----+-------+------+
 |       0.0        | 0.000187687687688 | 0.0 | 27977 | 5328 |
 | 0.0010000000475  |   0.999812312312  | 1.0 | 27977 | 5328 |
 | 0.00200000009499 |   0.999624624625  | 1.0 | 27977 | 5328 |
 | 0.00300000002608 |   0.999624624625  | 1.0 | 27977 | 5328 |
 | 0.00400000018999 |   0.999436936937  | 1.0 | 27977 | 5328 |
 | 0.00499999988824 |   0.999436936937  | 1.0 | 27977 | 5328 |
 | 0.00600000005215 |   0.999249249249  | 1.0 | 27977 | 5328 |
 | 0.00700000021607 |   0.999249249249  | 1.0 | 27977 | 5328 |
 | 0.00800000037998 |   0.999249249249  | 1.0 | 27977 | 5328 |
 | 0.00899999961257 |   0.999249249249  | 1.0 | 27977 | 5328 |
 +------------------+-------------------+-----+-------+------+
 [1001 rows x 

In [77]:
selected_words_model.show(view='Evaluation')

In [78]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

### What is the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model from the IPython Notebook from lecture? Save this result to answer the quiz at the end.

In [84]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [85]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [87]:
diaper_champ_reviews[0]['predicted_sentiment']

0.9999999372669541

### Now use the selected_words_model you learned using just the selected_words to predict the sentiment most positive review you found above

In [88]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.796940851290671]

### Why is the predicted_sentiment for the most positive review found using the model with all word counts (sentiment_model) much more positive than the one using only the selected_words (selected_words_model)? Hint: examine the text of this review, the extracted word counts for all words, and the word counts for each of the selected_words, and you will see what each model used to make its prediction. Save this result to answer the quiz at the end.

In [95]:
diaper_champ_reviews[['review','word_count','sel_word_count']][0]

{'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.',
 'sel_word_coun

### Answers to week 3 assignment
1. 'great'; 2. 'wow'; 3. 'love'; 4. 'terrible'; 5. 0.841 to 0.871; 6. 0.901 to 0.931;  7. 0.811 to 0.843 ; 8. 'The model learned using all words performed much better than the other two. The other two approaches performed about the same.'   ; 9. 0.9 to 1.0; 10. 0.7 to 0.8; 11. 'None of the selected_words appeared in the text of this review.'